<a href="https://colab.research.google.com/github/christine127/Portfolio_Recommendation_Project/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install finance-datareader
# !pip install opendartreader

In [2]:
!pip install pykrx

     |████████████████████████████████| 81 kB 4.1 MB/s 
     |████████████████████████████████| 60 kB 7.1 MB/s 
     |████████████████████████████████| 251 kB 41.9 MB/s 


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
# import FinanceDataReader as fdr
# import OpenDartReader
import datetime
from pykrx import stock
import warnings

warnings.filterwarnings('ignore')

In [23]:
# api_key = '5729a47b92ddd434526831ee99ff01f65e6434db'
# dart = OpenDartReader(api_key) 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# 전체 데이터
dataset = pd.read_excel('/content/drive/MyDrive/소회의실 프로젝트/Dataset.xlsx')
file = dataset.copy()
# 10명 샘플 데이터
# file = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/sample10.csv')
# file['고객구분코드'].unique()

In [19]:
print(file.columns)

Index(['고객구분코드', '주문일자', '고객성별구분코드', '동일나이군구분코드', '주소(시도)', '주문시간대', '상품번호',
       '상품명', '거래소구분코드', '매도매수구분코드', '주문구분코드', '실주문단가', '주문수량', '총체결수량',
       '총체결금액', '전일종가', '당일시가', '일중고가', '일중저가', '당일종가', '거래수량', '거래대금',
       '상한가여부', '하한가여부'],
      dtype='object')


In [24]:
## 주문일자
file['date'] = pd.to_datetime(file['주문일자'],format='%Y%m%d').astype(str)
file[['주문일자']] = file[[ '주문일자']].apply(pd.to_datetime, format='%Y%m%d')

In [25]:
##상품번호
file['code'] = file['상품번호'].str[-6:]

In [26]:
customer_info = pd.DataFrame(file.groupby(['고객구분코드','고객성별구분코드','동일나이군구분코드','주소(시도)'])['상품명'].nunique()).reset_index()
customer_info= customer_info.set_index('고객구분코드')
customer_info =customer_info.rename(columns={'상품명':'상품개수','주소(시도)':'주소'})

In [27]:
customer_info

,고객성별구분코드,동일나이군구분코드,주소,상품개수
고객구분코드,,,,
A00010020,1,20,서울,8
A0001035,2,40,경기,8
A00010416,1,30,서울,23
A00010516,2,30,서울,8
A00010708,1,50,울산,13
...,...,...,...,...
A0008961,1,20,경기,13
A0009016,2,50,경기,4
A0009145,1,30,부산,9


### 1. 거래수량 :총 매수수량
### 2. 거래액: 총 매수액


In [28]:
# #구매자별 매수금액
# customer_purchase = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드'])['총체결금액'].sum() 
# customer_amount = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드'])['총체결수량'].sum()

# # 코드 수정함
# customer_info['총매수금액']= customer_purchase
# customer_info['총매수수량'] = customer_amount


### 3. 투자상품의 종류별 비중(총매수금액을 가중치로) 
ETF, 대형주, 중형주, 소형주, KOSDAQ 


In [33]:
import math

kosdaq_tickers = stock.get_market_ticker_list("20200120", market="KOSDAQ")
kospi_tickers = stock.get_market_ticker_list("20200120", market="KOSPI")
kospi_caps  = stock.get_market_cap_by_ticker("20200120").loc[kospi_tickers,'시가총액'].sort_values(ascending= False)
kosdaq_caps  = stock.get_market_cap_by_ticker("20200120").loc[kosdaq_tickers,'시가총액'].sort_values(ascending= False)

kosdaq_tickers_2 = stock.get_market_ticker_list("20200331", market="KOSDAQ")
kospi_tickers_2 = stock.get_market_ticker_list("20200331", market="KOSPI")
kospi_caps_2  = stock.get_market_cap_by_ticker("20200331").loc[kospi_tickers_2,'시가총액'].sort_values(ascending= False)
kosdaq_caps_2  = stock.get_market_cap_by_ticker("20200331").loc[kosdaq_tickers_2,'시가총액'].sort_values(ascending= False)

kospi_caps = pd.concat([kospi_caps,kospi_caps_2],axis=1).mean(axis=1)
kosdaq_caps = pd.concat([kosdaq_caps,kosdaq_caps_2],axis=1).mean(axis=1)

all_caps = pd.concat([kospi_caps,kosdaq_caps],axis=1).mean(axis=1)
all_caps.sort_values(ascending=False, inplace=True)

large_cap = all_caps[:math.trunc(len(all_caps)*0.05)].index 
medium_cap = all_caps[math.trunc(len(all_caps)*0.05):math.trunc(len(all_caps)*0.3)].index
small_cap = all_caps[math.trunc(len(all_caps)*0.3):].index

file['stock_type']= ' '
file.loc[file['code'].isin(large_cap), 'stock_type'] ='large_cap'
file.loc[file['code'].isin(medium_cap), 'stock_type'] ='medium_cap'
file.loc[file['code'].isin(small_cap), 'stock_type'] ='small_cap'

In [39]:
#종목타입별 매수금액 비중

#구매자별 매수금액
customer_purchase = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드'])['총체결금액'].sum() 
customer_amount = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드'])['총체결수량'].sum()
#구매자별 상품별 매수금액
product_purchase = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드','code','stock_type'])['총체결금액'].sum().unstack().fillna(0)
#구매자별 종목타입별 매수금액 비중
type_purchase = product_purchase.groupby('고객구분코드').sum().apply(lambda x: x/customer_purchase )

customer_info = customer_info.join(type_purchase)
customer_info['총매수금액']= customer_purchase
customer_info['총매수수량'] = customer_amount

### 4. 투자한 섹터 개수
- 사용한 외부데이터 출처 : http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201 
주식 - 세부안내 - 업종별 분류현황

In [40]:
sector_kospi_200331 = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/Sector_KOSPI_200331.csv', encoding='cp949')
sector_kosdaq_200331 = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/Sector_KOSDAQ_200331.csv', encoding='cp949')
# new_stock = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/신규상장종목.csv', encoding='cp949')
# new_stock.rename(columns={'업종':'업종명'}, inplace=True)
sector_total = pd.concat([sector_kospi_200331.loc[:, ['종목코드', '업종명']], sector_kosdaq_200331.loc[:, ['종목코드', '업종명']]])
sector_total = sector_total.reset_index(drop = True)

#상장폐지, 신규상장 종목 추가
sector_total=sector_total.append({'종목코드' : '011160' , '업종명' : '건설업'} , ignore_index=True) # 3월 24일에 상장폐지된 코스피 두산건설
sector_total=sector_total.append({'종목코드' : '122800' , '업종명' : '금속'} , ignore_index=True) # 2월 12일에 상장폐지된 코스닥 썬테크놀로지스(썬텍)
sector_total=sector_total.append({'종목코드' : '341160' , '업종명' : '금융'} , ignore_index=True) # 1월 30일에 신규상장된 코스닥 하나금융15호스팩 - 작은 이상한점 : KRX종목리스트에는 이 종목이 안뜨네요. 일단 데이터에 있으니 추가.

In [41]:
sector_total.rename(columns={'종목코드':'code'}, inplace=True)
file = pd.merge(file, sector_total, how='left', left_on='code', right_on='code')

In [42]:
# 투자한 섹터 개수
sector_df = pd.DataFrame(file.groupby(['고객구분코드', '업종명']).size()).reset_index()
sector_df = pd.DataFrame(sector_df.groupby(['고객구분코드'])['업종명'].size()).reset_index()
sector_df.rename(columns={'업종명':'업종개수'}, inplace=True)

In [43]:
customer_info = pd.merge(customer_info, sector_df, on='고객구분코드')
customer_info

,고객구분코드,고객성별구분코드,동일나이군구분코드,주소,상품개수,총매수금액,총매수수량,large_cap,medium_cap,small_cap,업종개수
0,A00010020,1,20,서울,8,6665600,42,0.857147,0.142853,0.000000,6
1,A0001035,2,40,경기,8,20655870,376,0.830965,0.111725,0.057311,5
2,A00010416,1,30,서울,23,275445270,7020,0.876835,0.119032,0.004133,14
3,A00010516,2,30,서울,8,1855450,102,0.558786,0.388396,0.052817,5
4,A00010708,1,50,울산,13,25126330,6428,0.566957,0.246227,0.186816,7
...,...,...,...,...,...,...,...,...,...,...,...
97,A0008961,1,20,경기,13,768290,65,0.084018,0.704291,0.211691,9
98,A0009016,2,50,경기,4,3125500,160,0.520557,0.479443,0.000000,4
99,A0009145,1,30,부산,9,3512310,856,0.000000,0.141502,0.858498,5
100,A0009500,1,30,서울,14,5174710,304,0.774295,0.143776,0.081929,13


### 5. 종목보유기간: 매수-매도 기간 
거래량을 차감해서 계산하는 형태


In [44]:
# return 2가지
# 1. 매도된 주식이 언제 매수한 주식인지 계산
# 2. 매도가 안되고 남아 있는 주식의 인덱스 
# **** 거래일시가 sorting되어 있어야 작동 **** 
def first_sell_date_idx(df):
    a, b = np.array([]), np.array([])
    for idx, row in df.iterrows():
        if row['매도매수구분코드'] == 2:
            a = np.append(a, np.array(row['총체결수량']))
            b = np.append(b, np.array(0))
        elif row['매도매수구분코드'] == 1:
            b = np.append(b, np.array(row['총체결수량']))
            a = np.append(a, np.array(0))
            
#     print('a: ', a)
#     print('b: ', b)

    idx = np.array([])
    for i in range(len(a)):
        flag = True
        flag2 = True
        if b[i] == 0:
            idx = np.append(idx, np.array([-1]))
        else:
            for j in range(0, i):
                if(b[i] <= a[j]):                  
                    a[j] = a[j] - b[i]
                    b[i] = 0
                    if flag:
                        idx = np.append(idx, np.array([j]))
                        flag = False
                        flag2 = False
                    break
                elif (b[i] > a[j]) & (a[j] !=0):   
                    b[i] = b[i] - a[j]
                    a[j] = 0
                    if flag:
                        idx = np.append(idx, np.array([j]))
                        flag = False
                        flag2 = False
                        
                if((j == i-1) & (b[i] != 0)):
                    if flag2:
                        idx = np.append(idx, np.array([0]))
                    else:
                        idx[-1] = 0
                    b[i] == 0
                    
                
                   
    rest_index = np.where(a != 0)[0].tolist()
    return np.array(idx, dtype = np.int64).tolist(), rest_index

In [45]:
# 각각의 주문 보유기간 계산 하는 함수
# (고객번호, 계좌번호, 상품번호)별로 묶어서 계산
def buysell(df):
    
    cols = ['고객구분코드', '주문일자', '주문시간대', '상품번호', '상품명', 'stock_type', '매도매수구분코드', '실주문단가', '주문수량', '총체결수량', '총체결금액']
    combine_df = pd.DataFrame(columns= cols)
    df = df[['고객구분코드', '주문일자', '주문시간대', '상품번호', '상품명', 'stock_type','매도매수구분코드', '실주문단가', '주문수량', '총체결수량', '총체결금액']]
    
    for userID in tqdm(df['고객구분코드'].unique()):
        
        invest = df[(df['고객구분코드'] == userID)]

        # 매수매도되지 않은 주식의 보유기간 계산을 위한 기준일
        first_date = min(invest['주문일자'])
        # print(first_date)
        base_date = datetime.datetime(2020, 4, 1, 15, 0, 0)
        end_march = datetime.datetime.strptime('2020-03-15', '%Y-%m-%d')

        # (고객번호, 계좌번호)가 거래한 상품번호
        stock_code = invest['상품번호'].unique().tolist()

        # loop 한번 돌 때 마다 상품번호 끼리 묶음 
        for num in stock_code:
            temp = invest.loc[invest['상품번호'] == num]
            
            temp.loc[-1] =  {'고객구분코드': userID,
                           '주문일자': first_date,
                             '주문시간대':0,
                           '상품번호': num,
                           '매도매수구분코드': 2,
                            '총체결수량': 0}
            temp = temp.sort_values(by=['매도매수구분코드'], ascending=False).sort_values(by=['주문일자', '주문시간대'])

 

            """
            종목보유기간 계산
            - 매도시, 종목 보유 기간이 찍힘
            - 일반적으로 매도매수코드가 2일때(매수) NaN값을 가짐
            - 매도매수코드가 2인데 종목보유기간이 있는 행 --> 실제로 팔리지 않아 기준일(4월1일)에 팔았다고 가정하고 보유기간 계산
            - 3월 말 이후에 매수하고 매도되지않은 주식의 종목보유기간 --> -1 
            - 과거의 매수량보다 매도량이 많을 경우, 고객의 첫 구매일에 구매한 것으로 가정 --> 매수시점 idx = 0
            """
            # 총 매도수량
            sell_quantity = temp[temp['매도매수구분코드']==1]['총체결수량'].sum()
            # 총 매수수량
            buy_quantity = temp[temp['매도매수구분코드']==2]['총체결수량'].sum()

            # print('매도수량: ', sell_quantity, '    매수수량: ', buy_quantity)

            temp['매수시점idx'], rest_index =  first_sell_date_idx(temp)
            temp['종목보유기간'] = np.where(temp['매수시점idx'] == -1, np.nan, 
                                           (temp['주문일자'] - temp.iloc[temp['매수시점idx']]['주문일자'].values).dt.days)
            # 매도되지 못한 나머지
            for idx in rest_index:
                temp.iloc[idx, temp.columns.get_loc('종목보유기간')] = np.where(temp.iloc[idx, temp.columns.get_loc('주문일자')] > end_march, -1, 
                                                                          (base_date - temp.iloc[idx, temp.columns.get_loc('주문일자')]).days)
            
            
            temp.drop(index=temp.index[:1], axis=0, inplace=True)
            temp.drop('매수시점idx', axis=1, inplace=True)
            

            combine_df = pd.concat([combine_df, temp], axis=0)
        
  
    return combine_df        


In [50]:
def for_grouping(x):   
    if (~x.isin([-1])).sum() == 0: # 모든 종목보유기간이 -1 
        avg = -1
    else:
        avg = x[x >= 0].mean()
   
    return avg
    

# 고객의 평균 보유기간 계산 하는 함수    
def cal_term(df): 
    # cust_acc_set = df['고객구분코드'].unique()
    cust_cap_term = pd.DataFrame(columns=['고객구분코드', 'large_cap', 'medium_cap', 'small_cap'])
    # for cust, in cust_acc_set:
        # 매매 개별 종목보유기간 불러오기
    temp = buysell(df)[['고객구분코드', 'stock_type','종목보유기간']]

    # 그룹핑 하고 평균 내기, 모든 종목 보유기간이 -1일 경우 -1로 
    temp = temp[['고객구분코드', 'stock_type', '종목보유기간']]
    grouped = temp.groupby(['고객구분코드', 'stock_type'])
    temp = grouped.agg({'종목보유기간': for_grouping})

    temp = temp.pivot_table(index='고객구분코드', columns='stock_type', values='종목보유기간').reset_index()
    cust_cap_term = pd.concat([cust_cap_term, temp], ignore_index=True)
    cust_cap_term.columns = ['고객구분코드', 'large_cap_period', 'medium_cap_period', 'small_cap_period']

    cust_cap_term.fillna(0, inplace=True)
    return cust_cap_term
    # -1은 평균값으로 넣기 --> 매수는 했지만 매도는 없는 형태



In [52]:
pd.options.mode.chained_assignment = None
cal_term_df = cal_term(file)
cal_term_df = cal_term_df.replace(-1, np.nan)
cal_term_df.fillna(cal_term_df[cal_term_df[['large_cap_period', 'medium_cap_period', 'small_cap_period']] > 0].mean(), inplace=True)
# cal_term_df

In [54]:
customer_info = pd.merge(customer_info, cal_term_df, on='고객구분코드')
# customer_info

### 6. 수익률: (총매도금액/총매수금액 -1)
매도를 하지 않은 경우의 수익률은 어떻게 계산할지 조금 더 고민→ 4월 1일 매도 기준 

In [55]:
## 2020년 4월 1일 주가데이터 불러오기
stock_0401 = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/200401전종목가격.csv', encoding='cp949')

In [56]:
# 수익률 계산 
def cal_rate_of_return(df):
    cust_set = df['고객구분코드'].unique()
    cust_rate_return = pd.DataFrame(columns=['고객구분코드', '수익률'])
    
    for cust in cust_set:
        invest = df[df['고객구분코드'] == cust][['고객구분코드', '주문일자', '상품번호', '상품명', '매도매수구분코드','실주문단가', '주문수량', '총체결수량', '총체결금액']]
        combine_df = pd.DataFrame(columns= invest.columns)

        # (고객번호)가 거래한 상품번호
        stock_code = invest['상품번호'].unique().tolist()
        
        total_sell_price = 0
        total_buy_price = 0
        
        
        # loop 한번 돌 때 마다 상품번호 끼리 묶음 
        for num in stock_code:
            temp = invest.loc[invest['상품번호'] == num]

            first_date = min(temp['주문일자'])
            first_date = first_date.strftime('%Y%m%d')
            
            
            # ticker_code = temp['상품번호'].apply(lambda x: x[6:])
            ticker_code = temp['상품번호'].apply(lambda x: x[6:]).values[0]
            
            # 매도매수수량
            sell_quantity = temp[temp['매도매수구분코드']==1]['총체결수량'].sum()
            buy_quantity = temp[temp['매도매수구분코드']==2]['총체결수량'].sum()
            
            
            buy_price = temp[temp['매도매수구분코드']==2]['총체결금액'].sum()
            sell_price = temp[temp['매도매수구분코드']==1]['총체결금액'].sum()
            
            if buy_quantity >= sell_quantity:
                # 4월 1일 종가 가져오기
                # closing_price = stock.get_market_ohlcv_by_date("20200401", "20200401", ticker_code).iloc[0]['종가']
                closing_price = stock_0401[stock_0401['종목코드']== ticker_code].iloc[0]['종가'] # KRX사이트에서 받은 데이터에서 4월 1일 종가 가져오기
                sell_price = sell_price + (closing_price*(buy_quantity-sell_quantity))
            else:
                # 고객의 첫구매일 종가 가져오기
                # closing_price = stock.get_market_ohlcv_by_date(first_date, first_date, ticker_code).iloc[0]['종가']
                closing_price = df[df['code']== ticker_code].iloc[0]['당일종가'] # pykrx보다는 저희 원래 데이터가 정확하니 원래 데이터에서 종가 가져오는게 좋을것 같음
                buy_price = buy_price + (closing_price*(sell_quantity-buy_quantity))
                
            total_sell_price += sell_price
            total_buy_price += buy_price
        
        # 수익률
        rate_of_return = (total_sell_price/total_buy_price -1) * 100

        cust_rate_return = cust_rate_return.append({'고객구분코드':cust, '수익률':rate_of_return}, ignore_index=True)
    
    return cust_rate_return

In [57]:
customer_info = pd.merge(customer_info, cal_rate_of_return(file), on='고객구분코드')
# customer_info

In [58]:
customer_info= customer_info.set_index('고객구분코드')

## 상한가하한가 여부

In [59]:
def limit_price(df):
  df['상한가여부'] = df['상한가여부'].map(dict(Y=1, N=0))
  df['하한가여부'] = df['하한가여부'].map(dict(Y=1, N=0))

  df['상하한가여부'] = df['상한가여부'] | df['하한가여부']

  limit_price_df = pd.DataFrame(df.groupby('고객구분코드')['상하한가여부'].sum()/df.groupby('고객구분코드')['상하한가여부'].count())
  return limit_price_df

In [60]:
limit_df = limit_price(file)

In [61]:
limit_df

,상하한가여부
고객구분코드,
A00010020,0.000000
A0001035,0.000000
A00010416,0.202797
A00010516,0.047619
A00010708,0.000000
...,...
A0008961,0.387097
A0009016,0.000000
A0009145,0.615385


In [62]:
customer_info = customer_info.join(limit_df, on='고객구분코드')

In [63]:
customer_info

,고객성별구분코드,동일나이군구분코드,주소,상품개수,총매수금액,총매수수량,large_cap,medium_cap,small_cap,업종개수,large_cap_period,medium_cap_period,small_cap_period,수익률,상하한가여부
고객구분코드,,,,,,,,,,,,,,,
A00010020,1,20,서울,8,6665600,42,0.857147,0.142853,0.000000,6,12.125000,15.000000,0.000000,-3.215045,0.000000
A0001035,2,40,경기,8,20655870,376,0.830965,0.111725,0.057311,5,17.629881,13.185952,16.832704,-3.430357,0.000000
A00010416,1,30,서울,23,275445270,7020,0.876835,0.119032,0.004133,14,12.017241,12.600000,20.300000,-2.262515,0.202797
A00010516,2,30,서울,8,1855450,102,0.558786,0.388396,0.052817,5,29.333333,29.400000,25.000000,-6.904243,0.047619
A00010708,1,50,울산,13,25126330,6428,0.566957,0.246227,0.186816,7,14.625000,15.857143,15.000000,3.161780,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A0008961,1,20,경기,13,768290,65,0.084018,0.704291,0.211691,9,3.500000,6.600000,0.500000,7.355070,0.387097
A0009016,2,50,경기,4,3125500,160,0.520557,0.479443,0.000000,4,17.629881,13.185952,0.000000,-3.455447,0.000000
A0009145,1,30,부산,9,3512310,856,0.000000,0.141502,0.858498,5,0.000000,0.500000,0.000000,17.082984,0.615385


## PER, PBR, ROE, PSR, PEG
7. PER: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균 
8. PBR: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균
9. ROE: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균
10. PSR: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균
11. PEG: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균

## EPS, DIV
 12. EPS: 대 /중 / 소 / KOSDAQ 각각의 거래액기준 가중평균 - 주당순이익 - 기업 간 실적비교
 13. 배당률(DIV): 대 /중 / 소 / KOSDAQ 각각의 거래액기준 가중평균



In [64]:
jaemu = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/재무정보.csv')

In [65]:
## PER 
PER_df = jaemu.loc[:, ['날짜', '종목코드', 'PER']]
PER_df.rename(columns={'날짜':'date'}, inplace=True)
PER_df.rename(columns={'종목코드':'code'}, inplace=True)

# 원 데이터와 조인
file = pd.merge(file, PER_df, how='left',on=['code', 'date'])

file['PER'].fillna(0, inplace=True)

In [66]:
## PBR 
PBR_df = jaemu.loc[:, ['날짜', '종목코드', 'PBR']]
PBR_df.rename(columns={'날짜':'date'}, inplace=True)
PBR_df.rename(columns={'종목코드':'code'}, inplace=True)

# 원 데이터와 조인
file = pd.merge(file, PBR_df, how='left',on=['code', 'date'])

file['PBR'].fillna(0, inplace=True)

In [67]:
## ROE

file['ROE'] = file['PER']/ file['PBR']

file['ROE'].fillna(0, inplace=True)

In [68]:
## EPS 
EPS_df = jaemu.loc[:, ['날짜', '종목코드', 'EPS']]
EPS_df.rename(columns={'날짜':'date'}, inplace=True)
EPS_df.rename(columns={'종목코드':'code'}, inplace=True)

# 원 데이터와 조인
file = pd.merge(file, EPS_df, how='left',on=['code', 'date'])

file['EPS'].fillna(0, inplace=True)

In [69]:
## 배당률(DIV)
DIV_df = jaemu.loc[:, ['날짜', '종목코드', '배당수익률']]
DIV_df.rename(columns={'날짜':'date'}, inplace=True)
DIV_df.rename(columns={'종목코드':'code'}, inplace=True)
DIV_df.rename(columns={'배당수익률':'DIV'}, inplace=True)

# 원 데이터와 조인
file = pd.merge(file, DIV_df, how='left',on=['code', 'date'])

file['DIV'].fillna(0, inplace=True)

In [75]:
product_purchase_rate = product_purchase.apply(lambda x: x/customer_purchase )

for col in ['PER', 'PBR', 'ROE', 'EPS', 'DIV']:
  #고객의 상품별 EPS, DIV 평균
  product_col = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드','code','stock_type'])[col].mean().unstack().fillna(0)


  #고객별 종목타입별 EPS, DIV 가중평균
  col_rate = product_col* product_purchase_rate
  type_col = col_rate.groupby('고객구분코드').sum()

  # customer_info = customer_info.join(type_col, on='고객구분코드', rsuffix='_'+col)
  
  col_mean = pd.Series(np.sum(type_col* type_purchase, axis=1))
  customer_info[col]= col_mean

In [71]:
# ## PEG -- EPS가 (-)일 때 0으로 기록되어 있어서 직접 EPS부터 구해야 함
# stock.get_market_fundamental_by_date(file['주문일시'][38],file['주문일시'][38], file['code'][38])['EPS']
# stock.get_market_fundamental_by_date(file['주문일시'][38]-datetime.timedelta(weeks = 156),file['주문일시'][38]-datetime.timedelta(weeks = 156), file['code'][38])['EPS']
# # dart.finstate(file['상품번호'][38], '2020','11011')

### 14. 외국인 매매, 기관매매와의 매도/매수 일치 횟수 (%)

In [ ]:
#지분율
# stock.get_exhaustion_rates_of_foreign_investment_by_date(file['주문일시'][38],file['주문일시'][38], file['상품번호'][38])
# help(stock.get_exhaustion_rates_of_foreign_investment_by_date)
# stock.get_exhaustion_rates_of_foreign_investment_by_date("20200120", "20200120", "005930")

In [77]:
foreign = pd.read_csv("/content/drive/MyDrive/소회의실 프로젝트/외부데이터/외국인_순매수.csv")
foreign['날짜'] = foreign['날짜'].apply(pd.to_datetime, format('%Y-%m-%d'))
foreign.rename(columns={'거래대금_순매수':'외국인_순매수액'}, inplace= True)

inst = pd.read_csv("/content/drive/MyDrive/소회의실 프로젝트/외부데이터/기관_순매수.csv")
inst['날짜'] = inst['날짜'].apply(pd.to_datetime, format('%Y-%m-%d'))
inst.rename(columns={'거래대금_순매수':'기관_순매수액'}, inplace= True)

file = pd.merge(file, foreign, how='left', left_on=['주문일자','code'], right_on=['날짜','종목코드']).drop(['날짜','종목코드','종목명'], axis=1)
file = pd.merge(file, inst, how='left', left_on=['주문일자','code'], right_on=['날짜','종목코드']).drop(['날짜','종목코드','종목명'], axis=1)


In [78]:
file['외국인_순매수액'] = file['외국인_순매수액'].fillna(-1)
file['기관_순매수액'] = file['기관_순매수액'].fillna(-1)

file['외국인_매도매수구분코드'] = file['외국인_순매수액'].apply(lambda x: 1 if x<0 else (2 if x>0 else 0))
file['기관_매도매수구분코드'] = file['기관_순매수액'].apply(lambda x: 1 if x<0 else (2 if x>0 else 0))

# customer_info = customer_info.set_index('고객구분코드')
customer_info['매도매수일치_외국인'] = file.loc[file['외국인_매도매수구분코드'] ==file['매도매수구분코드'],['고객구분코드','매도매수구분코드']].groupby('고객구분코드').count().apply(lambda x: x/file.groupby('고객구분코드')['매도매수구분코드'].count()).fillna(0)
customer_info['매도매수일치_기관'] = file.loc[file['기관_매도매수구분코드'] ==file['매도매수구분코드'],['고객구분코드','매도매수구분코드']].groupby('고객구분코드').count().apply(lambda x: x/file.groupby('고객구분코드')['매도매수구분코드'].count()).fillna(0)

### ATR

In [79]:
ohlcv =pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/ohlcv_ATR.csv')

In [80]:
ohlcv['날짜'] =ohlcv['날짜'].apply(pd.to_datetime, format='%Y-%m-%d')
ohlcv_ATR = ohlcv[['종목코드','날짜','TR','ATR_7','ATR_14']].rename(columns = {'종목코드':'code','날짜':'주문일자'})
file = pd.merge(file, ohlcv_ATR, how='left',on = ['code','주문일자'])

In [86]:
for col in ['TR','ATR_7','ATR_14']:
  #고객의 상품별 TR(ATR) 평균
  product_col = file.groupby(['고객구분코드','code','stock_type'])[col].mean().unstack().fillna(0)
  
  #고객별 종목타입별 TR(ATR) 가중평균
  col_rate = product_col* product_purchase_rate
  type_col = col_rate.groupby('고객구분코드').sum()

  # customer_info = customer_info.join(type_col, on='고객구분코드', rsuffix='_'+col)
  customer_info[col]= col_mean
  col_mean = pd.Series(np.sum(type_col* type_purchase, axis=1))



14. 계좌개수: 계좌개수 
15. 시간대: ? -> 나중에 plot 시각화로 판단

18. 예약주문 횟수
19. 상한가,하한가 여부 횟수: 
주식을 산 날의 변동성을 보기위한 지표 

 
20. 정정/취소 횟수
21. 공모주 비중: 거래액 기준



## 시간대

In [ ]:
customer_info = customer_info.set_index('고객구분코드')

In [ ]:
time_percent = file.groupby('고객구분코드')['주문시간대'].value_counts().unstack().fillna(0).apply(lambda x: x/np.sum(file.groupby('고객구분코드')['주문시간대'].value_counts().unstack().fillna(0), axis=1))
customer_info = customer_info.join(time_percent,on='고객구분코드',lsuffix='시간대_').rename(columns=dict((i,'시간대_'+str(i)) for i in range(1,5)))
customer_info.join(time_percent,on='고객구분코드')

# 공모주 매수 비율

In [ ]:
new_stock = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/신규상장종목.csv', encoding='cp949')

In [ ]:
new_stock['신규종목여부'] = 1

In [ ]:
new_stock_df = new_stock.loc[:, ['종목코드', '신규종목여부']]
new_stock_df.rename(columns={'종목코드':'고객구분코드'}, inplace=True)
new_stock_df['고객구분코드'] = new_stock_df['고객구분코드'].astype(str)
file = pd.merge(file, new_stock_df, on='고객구분코드', how='left')
file['신규종목여부'].fillna(0, inplace=True)

In [ ]:
customer_amount = pd.DataFrame(customer_amount).reset_index()

In [ ]:
gongmo_amount = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드'])['신규종목여부'].sum()
gongmo_amount = pd.DataFrame(gongmo_amount).reset_index()
gongmo_amount['new_stock_rate'] = gongmo_amount['신규종목여부'] / customer_amount['총체결수량']
gongmo_amount = gongmo_amount.set_index('고객구분코드')

In [ ]:
customer_info = customer_info.join(gongmo_amount['new_stock_rate'], on='고객구분코드' )

,고객성별구분코드,동일나이군구분코드,주소,상품개수,new_stock_rate
고객구분코드,,,,,
A0001,1,30,경기,149,0.0
A00010,1,20,경남,10,0.0
A000100,2,40,서울,10,0.0
A0001000,1,30,광주,6,0.0
A00010000,1,20,경기,12,0.0
A00010002,2,20,대전,6,0.0
A00010003,1,20,울산,7,0.0
A00010004,2,50,대전,1,0.0
A00010005,1,20,대구,6,0.0
